In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
df = pd.read_csv('../input/vehicle-dataset-from-cardekho/car data.csv')

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df['Year'] = 2021 - df['Year']

In [ ]:
df.head(-5)

In [ ]:
pd.get_dummies(df['Seller_Type'])

In [ ]:
feature_metadata = {'seller_type_metadata':seller_type_metadata,
                    'fuel_type_metadata' : fuel_type_metadata,
                   'transmission_metadata' : transmission_metadata}

In [ ]:
metadata_file = open('feature_metadata.pickle','wb')
pickle.dump(feature_metadata,metadata_file)

In [ ]:
seller_type_metadata = {'Dealer' : 0, 'Individual' : 1}

In [ ]:
df['Seller_Type'] = df['Seller_Type'].map(seller_type_metadata)

In [ ]:
pd.get_dummies(df['Fuel_Type'])

In [ ]:
fuel_type_metadata = {'Petrol' : 2, 'Diesel' : 1, 'CNG' : 0}

In [ ]:
df['Fuel_Type'] = df['Fuel_Type'].map(fuel_type_metadata)

In [ ]:
df.head()

In [ ]:
pd.get_dummies(df['Transmission'])

In [ ]:
transmission_metadata = {'Automatic' : 1, 'Manual' : 0}

In [ ]:
df['Transmission'] = df['Transmission'].map(transmission_metadata)

In [ ]:
final_df = df.drop('Car_Name',axis=1)

In [ ]:
final_df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def plot_distribution(df,feature):
    plt.figure(figsize=[10,8])
    plt.title(feature)
    sns.distplot(df[feature])

In [ ]:
for feature in final_df.columns:
    plot_distribution(final_df,feature)

In [ ]:
def plot_box(df,feature):
    plt.figure(figsize=[10,8])
    plt.title(feature)
    sns.boxplot(df[feature])

In [ ]:
for feature in final_df.columns:
    plot_box(final_df,feature)

In [ ]:
plt.figure(figsize=[30,30])
sns.pairplot(final_df)

In [ ]:
x = final_df.drop('Selling_Price',axis=1)
y = final_df['Selling_Price']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)

In [ ]:
x_train_pca_components = pca.fit_transform(x_train)

In [ ]:
x_train_pca_components

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)]

In [ ]:
max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(5, 50, num = 10)]

min_samples_split = [2, 5, 10, 15, 25, 50, 75,100]

min_samples_leaf = [1, 2, 5, 7,10]

In [ ]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rf_random = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 20, verbose=1,cv = 5, random_state=42, n_jobs = -1)

### Using PCA Component Analysis

In [ ]:
rf_random.fit(x_train_pca_components,y_train)

In [ ]:
rf_random.best_score_

In [ ]:
rf_random.best_params_

In [ ]:
x_test_pca_components = pca.transform(x_test)

In [ ]:
predictions = rf_random.predict(x_test_pca_components)

In [ ]:
plt.figure(figsize=[10,8])
sns.distplot(y_test-predictions)
plt.title('Comparision')

In [ ]:
plt.figure(figsize=[10,8])
plt.scatter(y_test,predictions)
plt.title('Comparision')

In [ ]:
from sklearn import metrics

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

### Using Corealtion Matrix

In [ ]:
plt.figure(figsize=[20,20])
sns.heatmap(final_df.corr(),annot=True,cmap="RdYlGn")

In [ ]:
regressor_random = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 20, verbose=1,cv = 5, random_state=42, n_jobs = -1)

In [ ]:
regressor_random.fit(x_train,y_train)

In [ ]:
regressor_random.best_score_

In [ ]:
regressor_random.best_params_

In [ ]:
predictions = regressor_random.predict(x_test)

In [ ]:
plt.figure(figsize=[10,8])
sns.distplot(y_test-predictions)
plt.title('Comparision')
plt.savefig('distribution.png')

In [ ]:
plt.figure(figsize=[10,8])
plt.scatter(y_test,predictions)
plt.title('Comparision')
plt.savefig('scatter.png')

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
import pickle

In [ ]:
model_file = open('model.pickle','wb')
pickle.dump(regressor_random,model_file)